# Modele de regresie: Communities and Crime

Tătaru Dragoș-Cătălin, _grupa 10LF383_

In [1]:
import numpy as np
import pandas as pd

from IPython.display import display, HTML
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate,cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold


import sklearn
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.linear_model import Lasso, BayesianRidge, ElasticNet
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import uniform as sp_rand
import scipy.stats as stats

# Loading the data set

In [2]:
heads=['state', 'country','community','communityname','fold', 'population','householdsize','racepctblack','racePctWhite','racePctAsian','racePctHisp','agePct12t21','agePct12t29','agePct16t24','agePct65up','numbUrban','pctUrban','medIncome','pctWWage','pctWFarmSelf','pctWInvInc','pctWSocSec','pctWPubAs','pctWRetire','medFamInc','perCapInc', 'whitePerCap','blackPerCap','indianPerCap','AsianPerCap','OtherPerCap','HispPerCap','NumUnderPov','PctPopUnderPov','PctLess9thGrade','PctNotHSGrad','PctBSorMore','PctUnemployed','PctEmploy', 'PctEmplManu','PctEmplProfServ','PctOccupManu','PctOccupMgmtProf','MalePctDivorce','MalePctNevMarr','FemalePctDiv','TotalPctDiv','PersPerFam','PctFam2Par','PctKids2Par','PctYoungKids2Par','PctTeen2Par','PctWorkMomYoungKids','PctWorkMom','NumIlleg','PctIlleg','NumImmig','PctImmigRecent','PctImmigRec5','PctImmigRec8','PctImmigRec10','PctRecentImmig','PctRecImmig5','PctRecImmig8','PctRecImmig10','PctSpeakEnglOnly','PctNotSpeakEnglWell','PctLargHouseFam','PctLargHouseOccup','PersPerOccupHous','PersPerOwnOccHous','PersPerRentOccHous','PctPersOwnOccup','PctPersDenseHous','PctHousLess3BR','MedNumBR','HousVacant', 'PctHousOccup', 'PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','MedYrHousBuilt','PctHousNoPhone','PctWOFullPlumb','OwnOccLowQuart','OwnOccMedVal','OwnOccHiQuart','RentLowQ','RentMedian','RentHighQ','MedRent', 'MedRentPctHousInc','MedOwnCostPctInc','MedOwnCostPctIncNoMtg','NumInShelters', 'NumStreet','PctForeignBorn','PctBornSameState','PctSameHouse85','PctSameCity85','PctSameState85','LemasSwornFT','LemasSwFTPerPop','LemasSwFTFieldOps','LemasSwFTFieldPerPop', 'LemasTotalReq','LemasTotReqPerPop','PolicReqPerOffic','PolicPerPop','RacialMatchCommPol', 'PctPolicWhite','PctPolicBlack','PctPolicHisp','PctPolicAsian','PctPolicMinor','OfficAssgnDrugUnits','NumKindsDrugsSeiz','PolicAveOTWorked','LandArea','PopDens','PctUsePubTrans','PolicCars','PolicOperBudg','LemasPctPolicOnPatr','LemasGangUnitDeploy','LemasPctOfficDrugUn','PolicBudgPerPop','ViolentCrimesPerPop',]

# Replacing missing data types with NAN
missingValues = ["n/a", "na", "--"," ","?"]
data_crime = pd.read_csv("./Datasets/Communities and Crime/communities.data",sep=',',names=heads,na_values=missingValues)

display(data_crime)

,state,country,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,NaN,NaN,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.90,0.5,0.32,0.14,0.20
1,53,NaN,NaN,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.00,NaN,0.67
2,24,NaN,NaN,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.43
3,34,5.0,81440.0,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,NaN,NaN,NaN,NaN,0.00,NaN,0.12
4,42,95.0,6096.0,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,NaN,NaN,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,...,0.01,0.28,0.05,NaN,NaN,NaN,NaN,0.00,NaN,0.09
1990,6,NaN,NaN,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,...,0.02,0.37,0.20,NaN,NaN,NaN,NaN,0.00,NaN,0.45
1991,9,9.0,80070.0,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,...,0.08,0.32,0.18,0.08,0.06,0.78,0.0,0.91,0.28,0.23
1992,25,17.0,72600.0,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,...,0.03,0.38,0.33,0.02,0.02,0.79,0.0,0.22,0.18,0.19


In [3]:
display(data_crime.info())

# Afisam numarul de missind data in functie de coloana
df=pd.DataFrame(data_crime.isnull().sum())

# pentru a putea afisa toate coloanele nu doar limita pd
with pd.option_context('display.max_rows', None):display(df)

# Afisam numarul de missind data in total
print (data_crime.isnull().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Columns: 128 entries, state to ViolentCrimesPerPop
dtypes: float64(125), int64(2), object(1)
memory usage: 1.9+ MB


None

,0
state,0
country,1174
community,1177
communityname,0
fold,0
population,0
householdsize,0
racepctblack,0
racePctWhite,0
racePctAsian,0


39202


# Processing the dataset

In [4]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [5]:
# Stergem primele 5 coloane
columns = ['state','country','community','communityname','fold']
data_crime.drop(columns, inplace=True, axis=1)

# Stergem coloanele care au missing data
data_crime.dropna(axis=1,how="any", inplace=True)
display(data_crime)

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,0.30,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.09
1990,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,0.14,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.45
1991,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,0.54,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.23
1992,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,0.41,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.19


In [6]:
display(data_crime.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 100 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   population             1994 non-null   float64
 1   householdsize          1994 non-null   float64
 2   racepctblack           1994 non-null   float64
 3   racePctWhite           1994 non-null   float64
 4   racePctAsian           1994 non-null   float64
 5   racePctHisp            1994 non-null   float64
 6   agePct12t21            1994 non-null   float64
 7   agePct12t29            1994 non-null   float64
 8   agePct16t24            1994 non-null   float64
 9   agePct65up             1994 non-null   float64
 10  numbUrban              1994 non-null   float64
 11  pctUrban               1994 non-null   float64
 12  medIncome              1994 non-null   float64
 13  pctWWage               1994 non-null   float64
 14  pctWFarmSelf           1994 non-null   float64
 15  pct

None

# Splitting the dataset

In [7]:
newHead=data_crime.columns

X = data_crime.values[:, :99]
X = pd.DataFrame(X, columns=newHead[0:99])
y = data_crime.values[:, -1]
y = pd.DataFrame(y, columns=['ViolentCrimesPerPop'])

display(X)
display(y)

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.00,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.00,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.00,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.00,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.00,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,0.30,...,0.00,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00
1990,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,0.14,...,0.00,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00
1991,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,0.54,...,0.02,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91
1992,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,0.41,...,0.01,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22


,ViolentCrimesPerPop
0,0.20
1,0.67
2,0.43
3,0.12
4,0.03
...,...
1989,0.09
1990,0.45
1991,0.23
1992,0.19


# 1. Lasso Regression

In [8]:
pipe = Pipeline([('lss', Lasso(max_iter=50000))])
parameter_grid = { 'lss__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y, cv=k_fold,scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_lasso = pd.DataFrame(scores)

pd_scores_lasso.insert(0, "ModelName", ['Lasso_1', 'Lasso_2','Lasso_3', 'Lasso_4','Lasso_5']) 
display(pd_scores_lasso)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,Lasso_1,1.645372,0.002392,-0.141804,-0.143190,-0.035852,-0.035942,-0.118428,-0.121997
1,Lasso_2,1.499743,0.002610,-0.122075,-0.133772,-0.026584,-0.031816,-0.105551,-0.110669
2,Lasso_3,1.517855,0.002368,-0.102823,-0.091673,-0.022050,-0.017875,-0.064594,-0.060016
3,Lasso_4,1.509830,0.002236,-0.100719,-0.098518,-0.020826,-0.019951,-0.067370,-0.065080
4,Lasso_5,1.488843,0.002247,-0.124487,-0.128601,-0.028473,-0.029699,-0.102580,-0.105152


# 2. Ridge Regression

In [9]:
pipe = Pipeline([('rid', linear_model.Ridge())])
parameter_grid = { 'rid__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y, cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_ridge = pd.DataFrame(scores)

pd_scores_ridge.insert(0, "ModelName", ['Ridge_1', 'Ridge_2','Ridge_3', 'Ridge_4','Ridge_5']) 
display(pd_scores_ridge)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,Ridge_1,1.567584,0.002330,-0.089665,-0.091472,-0.016262,-0.017190,-0.060208,-0.063692
1,Ridge_2,1.479481,0.002244,-0.098620,-0.088824,-0.020108,-0.016267,-0.062325,-0.062231
2,Ridge_3,1.482990,0.002391,-0.093264,-0.091558,-0.016713,-0.017105,-0.072329,-0.062389
3,Ridge_4,1.492346,0.002190,-0.097587,-0.090316,-0.019978,-0.016416,-0.068213,-0.063695
4,Ridge_5,1.481705,0.002252,-0.093864,-0.090367,-0.018823,-0.016640,-0.061748,-0.063028


# 3. Elastic Net Regression

In [10]:
pipe = Pipeline([('enr', linear_model.ElasticNet())])
parameter_grid = { 'enr__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float),'enr__l1_ratio':np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}
k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=200)
scores = cross_validate(grid_search, X, y, cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_net_reg = pd.DataFrame(scores)

pd_scores_net_reg.insert(0, "ModelName", ['ElasticNet_1', 'ElasticNet_2','ElasticNet_3', 'ElasticNet_4','ElasticNet_5']) 
display(pd_scores_net_reg)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,ElasticNet_1,3.315015,0.002269,-0.093042,-0.094916,-0.018233,-0.018453,-0.063078,-0.062934
1,ElasticNet_2,3.040189,0.002249,-0.108478,-0.097891,-0.023539,-0.019686,-0.081255,-0.065458
2,ElasticNet_3,3.058739,0.002283,-0.091975,-0.092586,-0.017899,-0.017945,-0.058748,-0.061939
3,ElasticNet_4,3.032302,0.002225,-0.089620,-0.098397,-0.016043,-0.019909,-0.060199,-0.064404
4,ElasticNet_5,3.131592,0.002319,-0.099774,-0.091502,-0.021335,-0.017693,-0.065045,-0.060666


# 4. SGD Regression

In [11]:
pipe = Pipeline([('sgr', linear_model.SGDRegressor())])

parameter_grid = {'sgr__loss': ['squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'],
              'sgr__l1_ratio':  np.linspace(start=0, stop=1),
              'sgr__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float)}

k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y.to_numpy().ravel(), cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_sgdr = pd.DataFrame(scores)

pd_scores_sgdr.insert(0, "ModelName", ['SGD_1', 'SGD_2','SGD_3', 'SGD_4','SGD_5']) 
display(pd_scores_sgdr)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,SGD_1,1.688892,0.001177,-0.089875,-0.098990,-0.016871,-0.019675,-0.060065,-0.068606
1,SGD_2,1.576834,0.001353,-0.111043,-0.105052,-0.021051,-0.019463,-0.088192,-0.084198
2,SGD_3,1.540385,0.001168,-0.108292,-0.097014,-0.021765,-0.018800,-0.079941,-0.068876
3,SGD_4,1.543378,0.001129,-0.097834,-0.092674,-0.022457,-0.017744,-0.058376,-0.063807
4,SGD_5,1.540226,0.001148,-0.089257,-0.094434,-0.016112,-0.019409,-0.060995,-0.059789


# 5. LassoLars Regression

In [12]:
pipe = Pipeline([('lar',linear_model.LassoLars())])
parameter_grid = { 'lar__alpha': np.linspace(start=1e-5, stop=1, num=10000, dtype=float) }

k_fold = KFold(n_splits=5, shuffle=True)
grid_search =RandomizedSearchCV(estimator=pipe, param_distributions=parameter_grid,cv=KFold(3), n_iter=100)
scores = cross_validate(grid_search, X, y.to_numpy().ravel(), cv=k_fold, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True )
print("Scorurile rezultate in urma 5-fold cross validation")
pd_scores_laso = pd.DataFrame(scores)

pd_scores_laso.insert(0, "ModelName", ['LassoLars_1', 'LassoLars_2','LassoLars_3', 'LassoLars_4','LassoLars_5']) 
display(pd_scores_laso)

Scorurile rezultate in urma 5-fold cross validation


,ModelName,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,LassoLars_1,1.202815,0.001696,-0.168054,-0.181793,-0.049909,-0.055350,-0.140476,-0.160476
1,LassoLars_2,1.156848,0.001192,-0.097750,-0.104651,-0.019067,-0.021935,-0.066585,-0.071579
2,LassoLars_3,1.085992,0.001222,-0.183112,-0.176012,-0.056656,-0.053666,-0.154966,-0.145034
3,LassoLars_4,1.103128,0.001173,-0.102410,-0.092878,-0.023111,-0.017766,-0.062681,-0.063035
4,LassoLars_5,1.088311,0.001246,-0.181454,-0.177917,-0.054508,-0.054193,-0.159073,-0.150927


In [13]:
data_model_report=pd.DataFrame()
data_model_report=data_model_report.append([pd_scores_lasso,pd_scores_ridge,pd_scores_net_reg,pd_scores_sgdr,pd_scores_laso],ignore_index=True)
data_model_report.insert(1, " SearchStrategy ",'RandomizedSearchCV')

display(data_model_report)

,ModelName,SearchStrategy,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,Lasso_1,RandomizedSearchCV,1.645372,0.002392,-0.141804,-0.143190,-0.035852,-0.035942,-0.118428,-0.121997
1,Lasso_2,RandomizedSearchCV,1.499743,0.002610,-0.122075,-0.133772,-0.026584,-0.031816,-0.105551,-0.110669
2,Lasso_3,RandomizedSearchCV,1.517855,0.002368,-0.102823,-0.091673,-0.022050,-0.017875,-0.064594,-0.060016
3,Lasso_4,RandomizedSearchCV,1.509830,0.002236,-0.100719,-0.098518,-0.020826,-0.019951,-0.067370,-0.065080
4,Lasso_5,RandomizedSearchCV,1.488843,0.002247,-0.124487,-0.128601,-0.028473,-0.029699,-0.102580,-0.105152
5,Ridge_1,RandomizedSearchCV,1.567584,0.002330,-0.089665,-0.091472,-0.016262,-0.017190,-0.060208,-0.063692
6,Ridge_2,RandomizedSearchCV,1.479481,0.002244,-0.098620,-0.088824,-0.020108,-0.016267,-0.062325,-0.062231
7,Ridge_3,RandomizedSearchCV,1.482990,0.002391,-0.093264,-0.091558,-0.016713,-0.017105,-0.072329,-0.062389
8,Ridge_4,RandomizedSearchCV,1.492346,0.002190,-0.097587,-0.090316,-0.019978,-0.016416,-0.068213,-0.063695
9,Ridge_5,RandomizedSearchCV,1.481705,0.002252,-0.093864,-0.090367,-0.018823,-0.016640,-0.061748,-0.063028


# Generarea raportului

In [14]:
data_model_report['test_neg_mean_absolute_error'] = -data_model_report[['test_neg_mean_absolute_error']]
data_model_report[['train_neg_mean_absolute_error']] = -data_model_report[['train_neg_mean_absolute_error']]
data_model_report[['test_neg_mean_squared_error']] = -data_model_report[['test_neg_mean_squared_error']]
data_model_report[['train_neg_mean_squared_error']] = -data_model_report[['train_neg_mean_squared_error']]
data_model_report[['test_neg_median_absolute_error']] = -data_model_report[['test_neg_median_absolute_error']]
data_model_report[['train_neg_median_absolute_error']] = -data_model_report[['train_neg_median_absolute_error']]

data_model_report.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error',
                      'train_neg_mean_absolute_error':  'train_mean_absolute_error',
                      'test_neg_mean_squared_error':    'test_mean_squared_error',
                      'train_neg_mean_squared_error':   'train_mean_squared_error',
                      'test_neg_median_absolute_error': 'test_median_absolute_error', 
                      'train_neg_median_absolute_error':'train_median_absolute_error'}, inplace=True)

In [15]:
def highlight(s):
    is_max = s == s.max()
    is_min = s == s.min()
    styles = []
    for i in range(0, len(is_max)):
        if is_max[i]:
            styles.append('background-color: red')
        elif is_min[i]:
            styles.append('background-color: green')
        else:
            styles.append('')
    return styles

data_model_report_styled =data_model_report.style.apply(highlight, subset=['test_mean_absolute_error','train_mean_absolute_error','test_mean_squared_error','train_mean_squared_error','test_median_absolute_error','train_median_absolute_error'])

display(data_model_report_styled)

,ModelName,SearchStrategy,fit_time,score_time,test_mean_absolute_error,train_mean_absolute_error,test_mean_squared_error,train_mean_squared_error,test_median_absolute_error,train_median_absolute_error
0,Lasso_1,RandomizedSearchCV,1.645372,0.002392,0.141804,0.143190,0.035852,0.035942,0.118428,0.121997
1,Lasso_2,RandomizedSearchCV,1.499743,0.002610,0.122075,0.133772,0.026584,0.031816,0.105551,0.110669
2,Lasso_3,RandomizedSearchCV,1.517855,0.002368,0.102823,0.091673,0.022050,0.017875,0.064594,0.060016
3,Lasso_4,RandomizedSearchCV,1.509830,0.002236,0.100719,0.098518,0.020826,0.019951,0.067370,0.065080
4,Lasso_5,RandomizedSearchCV,1.488843,0.002247,0.124487,0.128601,0.028473,0.029699,0.102580,0.105152
5,Ridge_1,RandomizedSearchCV,1.567584,0.002330,0.089665,0.091472,0.016262,0.017190,0.060208,0.063692
6,Ridge_2,RandomizedSearchCV,1.479481,0.002244,0.098620,0.088824,0.020108,0.016267,0.062325,0.062231
7,Ridge_3,RandomizedSearchCV,1.482990,0.002391,0.093264,0.091558,0.016713,0.017105,0.072329,0.062389
8,Ridge_4,RandomizedSearchCV,1.492346,0.002190,0.097587,0.090316,0.019978,0.016416,0.068213,0.063695
9,Ridge_5,RandomizedSearchCV,1.481705,0.002252,0.093864,0.090367,0.018823,0.016640,0.061748,0.063028


# Exportarea raportului

In [16]:
html_str = """
<h1><u>Communities and Crime</u></h1>
"""
html_str = html_str + (data_model_report_styled.render())
html_file = open("./Reports/crime_reg_report.html", "w")
html_file.write(html_str)
html_file.close()